In [ ]:
using DrWatson
@quickactivate "BayesWorkshop2021"
include(joinpath(srcdir(), "setup.jl"));

## Hierarchical modeling

### Modeling varying intercepts for each building

Let's add a hierarchical intercept parameter, $\alpha_b$ at the building level to our model.

$$
\text{complaints}_{b,t} \sim \text{Neg-Binomial}(\lambda_{b,t}, \phi) \\
\lambda_{b,t}  = \exp{(\eta_{b,t})} \\
\eta_{b,t} = \mu_b + \beta \, {\rm traps}_{b,t} + \beta_{\rm super}\, {\rm super}_b + \text{log_sq_foot}_b \\
\mu_b \sim \text{Normal}(\alpha, \sigma_{\mu})
$$

In our Stan model, $\mu_b$ is the $b$-th element of the vector $\texttt{mu}$ which has one element per building.

One of our predictors varies only by building, so we can rewrite the above model more efficiently like so:

$$
\eta_{b,t} = \mu_b + \beta \, {\rm traps}_{b,t} + \text{log_sq_foot}_b\\
\mu_b \sim \text{Normal}(\alpha +  \beta_{\text{super}} \, \text{super}_b , \sigma_{\mu})
$$

We have more information at the building level as well, like the average age of the residents, the average age of the buildings, and the average per-apartment monthly rent so we can add that data into a matrix called `building_data`, which will have one row per building and four columns:

  * `live_in_super`
  * `age_of_building`
  * `average_tentant_age`
  * `monthly_average_rent`

We'll write the Stan model like:

$$
\eta_{b,t} = \alpha_b + \beta \, {\rm traps} + \text{log_sq_foot}\\
\mu \sim \text{Normal}(\alpha + \texttt{building_data} \, \zeta, \,\sigma_{\mu})
$$

### Prepare building data for hierarchical

We'll need to do some more data prep before we can fit our models. Firstly to use the building variable in Stan we will need to transform it from a factor variable to an integer variable.

In [ ]:
include(joinpath(srcdir(), "building_data.jl"))
building_data

### Fit the hierarchical model

Fit the model to data.

In [ ]:
using Turing

HalfNormal(args...) = truncated(Normal(args...), 1e-6, Inf)

function NegativeBinomial2(μ, ϕ)
    p = 1 / (1 + μ / ϕ)
    r = ϕ

    return NegativeBinomial(r, p)
end

@model function hier_NB_regression(; traps, live_in_super, log_sq_foot, building_idx, building_data, complaints=missing)
    J, K = size(building_data)

    sigma_mu ~ HalfNormal() # standard deviation of building-specific intercepts
    alpha ~ Normal(log(4), 1) # intercept of model for mu
    zeta ~ filldist(Normal(), K)  # coefficients on building-level predictors in model for mu
    beta ~ Normal(-0.25, 1) # coefficient on traps
    inv_phi ~ HalfNormal() # 1/phi (easier to think about prior for 1/phi instead of phi)

    # TODO: Implement
    mu ~ ...


    # Allocate if we're going to sample.
    complaints = complaints === missing ? Vector{Int}(undef, length(traps)) : complaints
    @. complaints ~ NegativeBinomial2(
        clamp(exp(mu[building_idx] + beta * traps + log_sq_foot), 1e-3, 1e5),
        clamp(inv(inv_phi), 1e-3, 100)
    )
    return (; sigma_mu, alpha, zeta, beta, mu, inv_phi, complaints)
end

In [ ]:
# Add the `log_sq_foot` as in 02.
transform!(pest_data, :total_sq_foot => ByRow(total_sq_foot -> log(total_sq_foot / 1e4)) => :log_sq_foot)

obs = (
    traps = pest_data.traps,
    live_in_super = pest_data.live_in_super,
    log_sq_foot = pest_data.log_sq_foot,
    building_idx = pest_data.building_idx,
    building_data = building_data
);
post = sample(hier_NB_regression(; obs..., complaints=pest_data.complaints), NUTS(), 1_000);

### Diagnostics

We get a bunch of warnings from Turing about divergent transitions, which is an indication that there may be regions of the posterior that have not been explored by the Markov chains.

Divergences are discussed in more detail in the course slides as well as the **bayesplot** (MCMC diagnostics vignette)[http://mc-stan.org/bayesplot/articles/visual-mcmc-diagnostics.html] and [*A Conceptual Introduction to Hamiltonian Monte Carlo*](https://arxiv.org/abs/1701.02434).

In this example we will see that we have divergent transitions because we need to reparameterize our model - i.e., we will retain the overall structure of the model, but transform some of the parameters so that it is easier for Stan to sample from the parameter space. Before we go through exactly how to do this reparameterization, we will first go through what indicates that this is something that reparameterization will resolve. We will go through:

1. Examining the fitted parameter values, including the effective sample size
2. Traceplots and scatterplots that reveal particular patterns in locations of the divergences.

First the chain:

In [ ]:
post

You can see that the effective samples are somewhat low for many of the parameters relative to the total number of samples. This alone isn't indicative of the need to reparameterize, but it indicates that we should look further at the trace plots and pairs plots. First let's look at the traceplots to see if the divergent transitions form a pattern.

In [ ]:
using StatsPlots
plot(post[[:sigma_mu]])

In [ ]:
# NOTE: If it fails because of argument error, just run until it doesn't.
# The prior is somewhat numerically unstable.
idata = to_arviz(
    hier_NB_regression(; obs...), # prior model
    hier_NB_regression(; obs..., complaints=pest_data.complaints), # conditioned model
    post # posterior samples
)

Looks as if the divergent parameters, the little black bars underneath the traceplots correspond to samples where the sampler gets stuck at one parameter value for $\sigma_\mu$.

In [ ]:
plot_trace(idata, var_names=["sigma_mu"])

And we can look at the joint distribution of `sigma_mu` with one of the means:

In [ ]:
let
    df = transform(
        DataFrame(post[[:sigma_mu, Symbol("mu[4]")]]),
        :sigma_mu => ByRow(log) => :log_sigma_mu,
        # `StatsPlots.@df` isn't to happy about `Symbol(...)`, so we
        # just rename it to `:mu4`.
        Symbol("mu[4]") => identity => :mu4
    )
    @df df cornerplot([:log_sigma_mu :mu4], compact=true)
end

What we have here is a cloud-like shape, with most of the divergences clustering towards the bottom. We'll see a bit later that we actually want this to look more like a funnel than a cloud, but the divergences are indicating that the sampler can't explore the narrowing neck of the funnel.

One way to see why we should expect some version of a funnel is to look at some simulations from the prior, which we can do without MCMC and thus with no risk of sampling problems:

In [ ]:
prior_samples = sample(hier_NB_regression(; obs..., complaints=pest_data.complaints), Prior(), 1_000);
let chain = prior_samples
    df = transform(
        DataFrame(chain[[:sigma_mu, Symbol("mu[4]")]]),
        :sigma_mu => ByRow(log) => :log_sigma_mu,
        # `StatsPlots.@df` isn't to happy about `Symbol(...)`, so we
        # just rename it to `:mu4`.
        Symbol("mu[4]") => identity => :mu4
    )
    @df df cornerplot([:log_sigma_mu :mu4], compact=true)
end

Of course, if the data is at all informative we shouldn't expect the posterior to look exactly like the prior. But unless the data is incredibly informative about the parameters and the posterior concentrates away from the narrow neck of the funnel, the sampler is going to have to confront the funnel geometry. (See the [Visual MCMC Diagnostics](http://mc-stan.org/bayesplot/articles/visual-mcmc-diagnostics.html) vignette for more on this.)

Another way to look at the divergences is via a parallel coordinates plot:

In [ ]:
plot_parallel(idata, var_names=["sigma_mu", "mu"])

Again, we see evidence that our problems concentrate when $\texttt{sigma_mu}$ is small.

### Reparameterize and recheck diagnostics

Instead, we should use the non-centered parameterization for $\mu_b$. We define a vector of auxiliary variables in the parameters block, $\texttt{mu_raw}$ that is given a $\text{Normal}(0, 1)$ prior in the model block. We then make $\texttt{mu}$ a transformed parameter: We can reparameterize the random intercept $\mu_b$, which is distributed:

$$
\mu_b \sim \text{Normal}(\alpha + \texttt{building_data} \, \zeta, \sigma_{\mu})
$$

This gives $\texttt{mu}$ a $\text{Normal}(\alpha + \texttt{building_data}\, \zeta, \sigma_\mu)$ distribution, but it decouples the dependence of the density of each element of $\texttt{mu}$ from $\texttt{sigma_mu}$ ($\sigma_\mu$). We will examine the effective sample size of the fitted model to see whether we've fixed the problem with our reparameterization.

Fit the model to the data.

In [ ]:
@model function hier_NB_regression_ncp(; traps, live_in_super, log_sq_foot, building_idx, building_data, complaints=missing)
    J, K = size(building_data)

    sigma_mu ~ HalfNormal()
    alpha ~ Normal(log(4), 1)
    zeta ~ filldist(Normal(), K)  # could also use informative priors on the different elements
    beta ~ Normal(-0.25, 1)
    inv_phi ~ HalfNormal()
    mu_raw ~ filldist(Normal(), J)

    mu  = alpha .+ building_data * zeta .+ sigma_mu .* mu_raw

    # Allocate if we're going to sample.
    complaints = complaints === missing ? Vector{Int}(undef, length(traps)) : complaints
    @. complaints ~ NegativeBinomial2(
        clamp(exp(mu[building_idx] + beta * traps + log_sq_foot), 1e-3, 1e5),
        clamp(inv(inv_phi), 1e-3, 100)
    )
    return (; sigma_mu, alpha, zeta, beta, mu, inv_phi, complaints)
end

Examining the fit of the new model

In [ ]:
post_ncp = sample(hier_NB_regression_ncp(; obs..., complaints=pest_data.complaints), NUTS(), 1_000);

This has improved the effective sample sizes of $\texttt{mu}$. We now run our usual posterior predictive checks.

In [ ]:
idata_ncp = to_arviz(
    hier_NB_regression_ncp(; obs...), # prior model
    hier_NB_regression_ncp(; obs..., complaints=pest_data.complaints), # conditioned model
    post_ncp # posterior samples
)

In [ ]:
# Now that `mu` isn't on the LHS of a `~` anymore, it won't be present in
# the resulting chain. Instead we need to extract it using `generated_quantities`:
generated = generated_quantities(
    hier_NB_regression_ncp(; obs..., complaints=pest_data.complaints),
    MCMCChains.get_sections(post_ncp, :parameters)
);

let
    # Extract `mu[4]` from `generated`.
    mu4 = mapreduce(vcat, generated) do g
        g.mu[4]
    end

    # Construct the `DataFrame`.
    df = select(
        DataFrame(post_ncp[[:sigma_mu]]),
        :sigma_mu => ByRow(log) => :log_sigma_mu,
    )
    # Add `mu[4]` to the `DataFrame.`
    df = DataFrame(:log_sigma_mu => df.log_sigma_mu, :mu4 => mu4)
    # Plot!
    @df df cornerplot([:log_sigma_mu :mu4], compact=true)
end

In [ ]:
plot_trace(idata_ncp, var_names=["sigma_mu"])

In [ ]:
plot_parallel(idata_ncp, var_names=["sigma_mu", "mu_raw"])

Notice how the numerical errors are now much reduced (depending on your random seed, they might be completely gone!).

In [ ]:
yrep = posterior_predictive(hier_NB_regression_ncp(; obs...), post_ncp)
ppc_dens_overlay(pest_data.complaints, yrep[1:200])

This looks quite nice. If we've captured the building-level means well, then the posterior distribution of means by building should match well with the observed means of the quantity of building complaints by month.

In [ ]:
ppc_stat_grouped(pest_data.complaints, yrep, group=pest_data.building_idx, stat="mean", binwidth=0.5)

We weren't doing terribly with the building-specific means before, but now they are all well-captured by our model. The model is also able to do a decent job estimating within-building variability:

In [ ]:
ppc_stat_grouped(pest_data.complaints, yrep, group=pest_data.building_idx, stat="sd", binwidth=0.5)

Predictions by number of traps:

In [ ]:
using RCall
ggplot = rimport("ggplot2")

ppc_intervals(pest_data.complaints, yrep, x=pest_data.traps) + ggplot.labs(x = "Number of traps", y = "Number of complaints")

Standardized residuals:

In [ ]:
# Convert into a matrix of size `(num_variables, num_samples)`
yrep_matrix = mapreduce(hcat, yrep) do y
    y.complaints
end;

qqnorm(dropdims(mean(yrep_matrix; dims=2); dims=2), qqline=:R)

Rootogram:

In [ ]:
ppc_rootogram(pest_data.complaints, yrep)

### Varying intercepts *and* varying slopes

We've gotten some new data that extends the number of time points for which we have observations for each building. This will let us explore how to expand the model a bit more with varying *slopes* in addition to the varying intercepts and also, later, also model temporal variation.

In [ ]:
let path = datadir("pest_data_longer_stan_dat.rds")
    R"""
    stan_dat_hier <- readRDS($path)
    """
end
pest_data_longer = rcopy(R"stan_dat_hier")
pest_data_longer = (; pairs(pest_data_longer)...)
pest_data_longer = merge(
    pest_data_longer, 
    (complaints = Int.(pest_data_longer.complaints), traps = Int.(pest_data_longer.traps))
)

Perhaps if the levels of complaints differ by building, the coefficient for the effect of traps on building does too. We can add this to our model and observe the fit.

$$
\text{complaints}_{b,t} \sim \text{Neg-Binomial}(\lambda_{b,t}, \phi)  \\
\lambda_{b,t} = \exp{(\eta_{b,t})}\\
\eta_{b,t} = \mu_b + \kappa_b \, \texttt{traps}_{b,t} + \text{log_sq_foot}_b \\
\mu_b \sim \text{Normal}(\alpha + \texttt{building_data} \, \zeta, \sigma_{\mu}) \\
\kappa_b \sim \text{Normal}(\beta + \texttt{building_data} \, \gamma, \sigma_{\kappa})
$$

Fit the model to data and extract the posterior draws needed for our posterior predictive checks.

In [ ]:
@model function hier_NB_regression_ncp_slopes(; traps, log_sq_foot, building_idx, building_data, complaints)
    J, K = size(building_data)

    # TODO: Implement
    kappa_raw ~ ...
    sigma_kappa ~ ...
    gamma ~ ...

    sigma_mu ~ HalfNormal()
    alpha ~ Normal(log(4), 1)
    zeta ~ filldist(Normal(), K)  # could also use informative priors on the different elements
    beta ~ Normal(-0.25, 1)
    inv_phi ~ HalfNormal()
    mu_raw ~ filldist(Normal(), J)

    mu  = alpha .+ building_data * zeta .+ sigma_mu * mu_raw
    # TODO Implement
    kappa = ...

    # Allocate if we're going to sample.
    complaints = complaints === missing ? Vector{Int}(undef, length(traps)) : complaints
    @. complaints ~ NegativeBinomial2(
        exp(mu[building_idx] + kappa[building_idx] * traps + log_sq_foot),
        inv(inv_phi)
    )
    return (; sigma_mu, sigma_kappa, gamma, alpha, zeta, beta, mu, kappa, inv_phi, complaints)
end

In [ ]:
obs = (
    traps = pest_data_longer.traps,
    log_sq_foot = pest_data_longer.log_sq_foot,
    building_idx = pest_data_longer.building_idx,
    building_data = building_data
);

post_ncp_slopes = sample(hier_NB_regression_ncp_slopes(; obs..., complaints=pest_data_longer.complaints), NUTS(), 1_000)

To see if the model infers building-to-building differences in, we can plot a histogram of our marginal posterior distribution for `sigma_kappa`.

In [ ]:
plot(post_ncp_slopes[[:sigma_kappa]])

In [ ]:
plot(MCMCChains.group(post_ncp_slopes, :kappa_raw))

While the model can't specifically rule out zero from the posterior, it does have mass at small non-zero numbers, so we should leave in the hierarchy over $\texttt{kappa}$. Plotting the marginal data density again, we can see the model still looks well calibrated.

In [ ]:
yrep = posterior_predictive(hier_NB_regression_ncp_slopes(; obs...), post_ncp_slopes)

In [ ]:
ppc_dens_overlay(pest_data_longer.complaints, yrep[1:200])

In [ ]:
ppc_stat_grouped(pest_data_longer.complaints, yrep, group=pest_data_longer.building_idx, stat="mean", binwidth=0.5)

In [ ]:
ppc_stat_grouped(pest_data_longer.complaints, yrep, group=pest_data_longer.building_idx, stat="sd", binwidth=0.5)

In [ ]:
ppc_intervals(pest_data_longer.complaints, yrep, x=pest_data_longer.traps) + ggplot.labs(x = "Number of traps", y = "Number of complaints")

In [ ]:
ppc_rootogram(pest_data_longer.complaints, yrep)